In [1]:
%cd ..

/home/melek/workspace/vscode/nb_gpu_trainer


In [29]:
from nbtr.model.gpt2 import GPT
from huggingface_hub import HfApi, login
from transformers.utils import cached_file
from nbtr.trainer import TrainerConfig
from nbtr.tokenizer.tokenizer import Tokenizer
from torch.nn import functional as F
import torch

In [7]:
login()

In [8]:
api = HfApi()

In [10]:
trainer_cfg_file = "config/news_trainer.yml"
model_cfg_file = "config/news_model.yml"

In [14]:
def load_config(config_file)->TrainerConfig:
    import yaml

    with open(config_file) as f:
        try:
            doc = yaml.safe_load(f)
        except yaml.YAMLError as exc:
            print(exc)
            exit(1)

    return TrainerConfig(**doc)

trainer_cfg = load_config(trainer_cfg_file)

In [15]:
trainer_cfg

TrainerConfig(seq_length=1024, gradient_accumulation_steps=1, batch_size=32, ds_repo_id='habanoz/haber-90k-gpt-text', data_dir='haber-90k-data', warmup_iters=100, learning_rate=0.001, lr_decay_iters=6000, max_iters=6000, min_lr=0.0001, weight_decay=0.1, beta1=0.9, beta2=0.99, compile=False, decay_lr=True, seed=145, log_interval=10, eval_interval=250, eval_iters=200, out_dir='haber-90k-gpt', wandb_log=True, wandb_project='NB-Haber-GPT-Training', wandb_run_name='haber-90k-gpt-v1.1', wandb_run_id='1721342907', grad_norm_clip=1.0, repo_id='habanoz/haber-90k-gpt-v1.1', dtype='float16')

In [11]:
model = GPT.from_config(model_cfg_file)

number of parameters: 13.77M


In [18]:
model_state_file = cached_file(trainer_cfg.repo_id, 'pytorch_model.bin', _raise_exceptions_for_missing_entries=True)
model_state = torch.load(model_state_file, map_location=torch.device('cpu') )

In [19]:
model.load_state_dict(model_state)

<All keys matched successfully>

In [20]:
model

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(8192, 384)
    (wpe): Embedding(1024, 384)
    (drop): Dropout(p=0.2, inplace=False)
    (h): ModuleList(
      (0-5): 6 x Block(
        (ln_1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=384, out_features=1152, bias=False)
          (c_proj): Linear(in_features=384, out_features=384, bias=False)
          (attn_dropout): Dropout(p=0.2, inplace=False)
          (resid_dropout): Dropout(p=0.2, inplace=False)
        )
        (ln_2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=384, out_features=1536, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=1536, out_features=384, bias=False)
          (dropout): Dropout(p=0.2, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head

In [21]:
model.eval()

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(8192, 384)
    (wpe): Embedding(1024, 384)
    (drop): Dropout(p=0.2, inplace=False)
    (h): ModuleList(
      (0-5): 6 x Block(
        (ln_1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=384, out_features=1152, bias=False)
          (c_proj): Linear(in_features=384, out_features=384, bias=False)
          (attn_dropout): Dropout(p=0.2, inplace=False)
          (resid_dropout): Dropout(p=0.2, inplace=False)
        )
        (ln_2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=384, out_features=1536, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=1536, out_features=384, bias=False)
          (dropout): Dropout(p=0.2, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head

In [36]:
_tokenizer = Tokenizer.from_pretrained(trainer_cfg.repo_id)

In [37]:
_tokenizer.encode("# Borsa güne yükselişle başladı")

{'input_ids': [1, 8108, 8141, 6080, 6610, 4559, 42, 961, 2]}

In [73]:
prompt = torch.tensor(_tokenizer.encode("# Turkcell’den bayramda yurt dışı müjdesi")['input_ids']).view(1, -1)
prompt = prompt[:,:-1]
for i in range(512):
    logits,_ = model(prompt)
    logits = logits[:, -1, :]
    probs = F.softmax(logits, dim=1)
    tok = torch.multinomial(probs, 1, replacement=True)
    
    if tok.item() == 2:
        break

    prompt = torch.cat((prompt , tok), dim=-1)

text = _tokenizer.decode(prompt.tolist()[0])

print(text)

# Turkcell’den bayramda yurt dışı müjdesi ## Özet Turkcell Global Çocuk Koleksiyonu ve PSA Türkiye, 8-8 Mayıs arası gerçekleştirilecek "Hemen İçin Vuhan Yol Kampı"nda, başkent Turkcell'de yapılacak, 28-8 Mayıs arası gerçekleşecekleri yeni tip koronavirüs (Kovid-19) salgını sebebiyle çevrim içi arabaların çevrim içi duyurulduğu hesaplandığı açıklandı ## İçerik Turkcell'den yapılan açıklamaya göre, Turkcell’in online eğitim tarihinde ilk kez ailesinin katkısı, Turkcell'in dijitalleştiği bir iş ortağı olma vizyonuyla iş hayatında oluşmaya devam ediyor. Y Mevcut adımların faaliyetlerine en yakın bilgi veren Turkcell Global Çocuk Koleksiyonu ve PSA Türkiye, 8-9 Mayıs arası gerçekleşecek. Turkcell Global Çocuk Koleksiyonu’na 10-8 Mayıs arası gerçekleştirilecekleri yeni tip koronavirüs (Kovid-19) salgını sebebiyle çevrim içi alışverişlerde Garanti BBVA Grubunun avantajları ile yanıtlandı. Turkcell,  ⁇ R kodları ve TWF “Yılın Sosyal Güvenlik Dayanışma Kampı”nda, Turkcell Dijital PSA Türkiye, 7

In [74]:
from IPython.display import display, Markdown

In [75]:
display(Markdown(text))

# Turkcell’den bayramda yurt dışı müjdesi ## Özet Turkcell Global Çocuk Koleksiyonu ve PSA Türkiye, 8-8 Mayıs arası gerçekleştirilecek "Hemen İçin Vuhan Yol Kampı"nda, başkent Turkcell'de yapılacak, 28-8 Mayıs arası gerçekleşecekleri yeni tip koronavirüs (Kovid-19) salgını sebebiyle çevrim içi arabaların çevrim içi duyurulduğu hesaplandığı açıklandı ## İçerik Turkcell'den yapılan açıklamaya göre, Turkcell’in online eğitim tarihinde ilk kez ailesinin katkısı, Turkcell'in dijitalleştiği bir iş ortağı olma vizyonuyla iş hayatında oluşmaya devam ediyor. Y Mevcut adımların faaliyetlerine en yakın bilgi veren Turkcell Global Çocuk Koleksiyonu ve PSA Türkiye, 8-9 Mayıs arası gerçekleşecek. Turkcell Global Çocuk Koleksiyonu’na 10-8 Mayıs arası gerçekleştirilecekleri yeni tip koronavirüs (Kovid-19) salgını sebebiyle çevrim içi alışverişlerde Garanti BBVA Grubunun avantajları ile yanıtlandı. Turkcell,  ⁇ R kodları ve TWF “Yılın Sosyal Güvenlik Dayanışma Kampı”nda, Turkcell Dijital PSA Türkiye, 7/24 Mayıs tarihlerinde İstanbul Mobil Okrugöz Bisiklet Fonu ve Bisiklet Fonu sosyal pasaportlarına Turkcell tarafından uygulanan "Hemen İçin Vuhan Yol Kampı"nda, dijital başka alanlarda ve en çok okuyucuyla buluştı. Turkcell’in sunduğu Dijital PSA Türkiye, 8-8 Mayıs arasındaki anlaşmayı 4 Mayıs’a kadar tamamlayacak. Turkcell Global Çocuk Koleksiyonu ve PSA’nın web sitesi arasında kullanıcılar, gençler, isterseprobassiyonunu mobil platformlarda, performans gösteren bağlı sensörler gibi popülerliğine ulaşmada da önem veren Turkcell Kurumsal Çocuk Koleksiyonu ve PSA’nın dijital denilebilecekleri yeni programda Turkcell'in dijitalleştirici güvenlik projeleri üstlendi. Turkcell Global Çocuk Koleksiyonu ve PSA Türkiye’nin Yurtiçi üyesi olacak LG InP, kullanıcıların bir evle olan kolayca karşılanacağı hediye çekleriyle birlikte sunuluyor. Turkcell tüm Turkcell Cargo, haziran ve içinde gerçekleştirilen tüm Turkcell’in salgın sürecine üye olma vizyonuyla adım adım adım adımı atıyor.

In [65]:
from datasets import load_dataset

In [66]:
ds = load_dataset("habanoz/haber-90k-gpt-text")

In [67]:
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 81708
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 9079
    })
})

In [71]:
print(ds['train']['text'][0])

# Turkcell’den bayramda yurt dışı müjdesi

## Özet

Turkcell, tarifesi ne olursa olsun Turkcell, 5-18 Ağustos'ta yurt dışında olacak bireysel ve kurumsal faturalı tüm müşterilerine 1GB internetin yanı sıra 60 dakika görüşme veya 60 SMS hediye ediyor

## İçerik

Şirketten yapılan açıklamaya göre, müşterilerini sadece Türkiye’de değil, dünyanın her yerinde en iyi hizmetle buluşturma amacıyla çalışan Turkcell, Kurban Bayramı'nda yurt dışına çıkacak müşterilerine özel bir hediye sunuyor.

Bireysel ve kurumsal faturalı tüm Turkcell’liler tarifelerinden bağımsız olarak bayram döneminde yurt dışında yapacakları 1GB’lık internet kullanımının yanı sıra 60 dakika görüşme (arama/aranma) veya 60 SMS’i bayram hediyesi olarak ücretsiz kullanabilecek.

88 ülkeyi kapsayan Kurban Bayramı hediyesiyle, Turkcell’liler rahat bir şekilde iletişim kurabilecek. 5-18 Ağustos'ta yurt dışında bulunacaklara otomatik olarak tanımlanacak olan hediyeyle Turkcell müşterileri sevdikleriyle Türkiye’deki gibi bayramlaşa

In [72]:
display(Markdown(ds['train']['text'][0]))

# Turkcell’den bayramda yurt dışı müjdesi

## Özet

Turkcell, tarifesi ne olursa olsun Turkcell, 5-18 Ağustos'ta yurt dışında olacak bireysel ve kurumsal faturalı tüm müşterilerine 1GB internetin yanı sıra 60 dakika görüşme veya 60 SMS hediye ediyor

## İçerik

Şirketten yapılan açıklamaya göre, müşterilerini sadece Türkiye’de değil, dünyanın her yerinde en iyi hizmetle buluşturma amacıyla çalışan Turkcell, Kurban Bayramı'nda yurt dışına çıkacak müşterilerine özel bir hediye sunuyor.

Bireysel ve kurumsal faturalı tüm Turkcell’liler tarifelerinden bağımsız olarak bayram döneminde yurt dışında yapacakları 1GB’lık internet kullanımının yanı sıra 60 dakika görüşme (arama/aranma) veya 60 SMS’i bayram hediyesi olarak ücretsiz kullanabilecek.

88 ülkeyi kapsayan Kurban Bayramı hediyesiyle, Turkcell’liler rahat bir şekilde iletişim kurabilecek. 5-18 Ağustos'ta yurt dışında bulunacaklara otomatik olarak tanımlanacak olan hediyeyle Turkcell müşterileri sevdikleriyle Türkiye’deki gibi bayramlaşacak.

-“Turkcell’liler bayramın coşkusunu yurt dışında da yaşayacak”

Açıklamada görüşlerine yer verilen Turkcell Genel Müdür Yardımcısı Ömer Barbaros Yiş, şunları kaydetti:

“Turkcell olarak sadece yurt içinde değil yurt dışında da müşterilerimizin yanındayız. Ülkemiz ve insanlarımızın için son derece manevi anlamlar taşıyan Kurban Bayramı’nda müşterilerimize yakınlarıyla rahatça bayramlaşabilmeleri için bugüne kadar benzeri olmayan bir hediye sunuyoruz. Bu hediyemiz sayesinde Turkcell’liler gerek internet üzerinden gerekse konuşarak bayramın coşkusunu dünyanın dört bir yanına taşıyacak. Bu vesileyle tüm Türkiye’nin bayramını şimdiden kutlarız.”

Yurt dışındaki ilk kullanımda devreye girecek ve 1 saat geçerli olacak hediyenin kullanılabileceği ülkeler şöyle:

ABD, Almanya, Arnavutluk, Avustralya, Avusturya, Azerbaycan, Bangladeş, Belçika, Beyaz Rusya, Birleşik Arap Emirlikleri, Bosna Hersek, Brezilya, Bruney, Bulgaristan, Cebelitarık, Çek Cumhuriyeti, Çin, Danimarka, Ekvator, Ermenistan, Estonya, Faroe Adaları, Fiji, Filistin, Finlandiya, Fransa, Guatemala, Güney Afrika, Güney Kore, Gürcistan, Hırvatistan, Hollanda, Hong Kong, Irak, İngiltere, İran, İrlanda, İspanya, İsrail, İsveç, İsviçre, İtalya, İzlanda, Kanada, Karadağ, Katar, Kazakistan, Kıbrıs, Kırgızistan, Kosova, Kosta Rika, Letonya, Lihtenştayn, Litvanya, Lüksemburg, Macaristan, Makao, Makedonya, Malezya, Malta, Mısır, Moldova, Monako, Montserrat, Nepal, Nijer, Norveç, Panama, Polonya, Portekiz, Porto Riko, Romanya, Rusya, San Marino, Sırbistan, Singapur, Slovakya, Slovenya, Sri Lanka, Suudi Arabistan, Tacikistan, Tayland, Tayvan, Tunus, Ukrayna, Uruguay, Yeni Zelanda, Yunanistan.